In [1]:
%%capture
!pip install mordred
!pip install rdkit

In [2]:
# Importing Libraries
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, AllChem

import mordred
from mordred import Calculator, descriptors

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [5]:
df_final = pd.read_csv('delaney_8_des.csv')
df_final.head()

,FilterItLogS,Lipinski,SIC0,RNCG,RPCG,ATS0Z,PEOE_VSA6,AATS0i,measured log(solubility:mol/L)
0,-2.790326,1,0.500000,0.330754,0.551252,1230,34.80282,162.007716,-2.18
1,-2.129911,1,0.520426,0.311690,0.695189,942,34.80282,164.102494,-2.00
2,-2.433986,1,0.500000,0.250000,0.335426,1230,0.00000,162.007716,-1.74
3,-2.147371,1,0.520426,0.372917,0.364050,942,0.00000,164.102494,-1.48
4,-2.664600,1,0.520426,0.277752,0.533222,1182,23.20188,208.591109,-3.04


In [6]:
y = df_final['measured log(solubility:mol/L)']
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(df_final.iloc[:, :-1]), columns = df_final.iloc[:, :-1].columns)

In [7]:
X.head()

,FilterItLogS,Lipinski,SIC0,RNCG,RPCG,ATS0Z,PEOE_VSA6,AATS0i
0,0.021772,0.332685,1.784579,0.033750,3.187876,0.534794,0.443669,-0.077580
1,0.376467,0.332685,1.993944,-0.070165,4.534268,0.121242,0.443669,0.215509
2,0.213155,0.332685,1.784579,-0.406439,1.169029,0.534794,-1.168915,-0.077580
3,0.367089,0.332685,1.993944,0.263581,1.436776,0.121242,-1.168915,0.215509
4,0.089296,0.332685,1.993944,-0.255165,3.019216,0.465869,-0.093859,6.440094


In [8]:
y

0      -2.180
1      -2.000
2      -1.740
3      -1.480
4      -3.040
        ...  
1139    1.144
1140   -4.925
1141   -3.893
1142   -3.790
1143   -2.581
Name: measured log(solubility:mol/L), Length: 1144, dtype: float64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=45)

In [15]:
rf = RandomForestRegressor(bootstrap= True,
 max_depth= 45,
 max_features= 'log2',
 min_samples_leaf= 2,
 min_samples_split= 5,
 n_estimators= 800,
random_state=0)
rf.fit(X_train, y_train)
print(f'The r2 score for train set is : {rf.score(X_train, y_train)}')
print(f'The r2 score for test set is : {rf.score(X_test, y_test)}')

The r2 score for train set is : 0.9664117294739257
The r2 score for test set is : 0.8638248299806615


In [23]:
import pickle
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)
with open('model_rf', 'wb') as f:
    pickle.dump(rf, f)

## **Predict Solubility from SMILES**

In [16]:
X.columns

Index(['FilterItLogS', 'Lipinski', 'SIC0', 'RNCG', 'RPCG', 'ATS0Z',
       'PEOE_VSA6', 'AATS0i'],
      dtype='object')

**Prediction for a single SMILES**

In [17]:
def predict_sol(smile):
  mol = Chem.MolFromSmiles(smile)
  mol = Chem.AddHs(mol)
  AllChem.EmbedMolecule(mol)

  df_mol = pd.DataFrame(data = [mol], columns=(['mol']), dtype='object')
  calc = Calculator(descriptors, ignore_3D=False)
  desc = calc.pandas(df_mol['mol'])
  desc_8 = desc[['FilterItLogS', 'Lipinski', 'SIC0', 'RNCG', 'RPCG', 'ATS0Z',
        'PEOE_VSA6', 'AATS0i']]
  X_test = scaler.transform(desc_8)
  predict = rf.predict(X_test)
  print(f'The Predicted Solubility is {predict[0]}')



In [18]:
predict_sol('OCC1=C(O)C=C(O)C=C1')

100%|██████████| 1/1 [00:00<00:00,  3.23it/s]

The Predicted Solubility is -1.2217895199765518


**Predictions for a list of SMILES**

In [21]:
def predict_sol_smiles(smiles):
  mols = []
  for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    mols.append(mol)
  df_mol = pd.DataFrame(data = mols, columns=(['mol']), dtype='object')
  calc = Calculator(descriptors, ignore_3D=False)
  desc = calc.pandas(df_mol['mol'])
  desc_8 = desc[['FilterItLogS', 'Lipinski', 'SIC0', 'RNCG', 'RPCG', 'ATS0Z',
        'PEOE_VSA6', 'AATS0i']]
  X_test = scaler.transform(desc_8)
  predict = rf.predict(X_test)
  print(f'The Predicted Solubilities is {predict}')


In [22]:
smiles = ['OCC1=C(O)C=C(O)C=C1', 'NCC1=C(N)C=C(N)C=C1', 'CCC1=C(C)C=C(C)C=C1']
predict_sol_smiles(smiles)

100%|██████████| 3/3 [00:00<00:00,  4.23it/s]

The Predicted Solubilities is [-1.22178952 -1.56682183 -4.09530883]
